In [1]:
# Import the required libraries
from google.colab import drive
import pandas as pd
import numpy as np
import csv

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path = '/content/drive/MyDrive/557 Project - Group 44/Top100stocks_indicator.csv'
df_all = pd.read_csv(file_path)

In [4]:
# df.head
unique_names = df_all['SYMBOL'].unique()

In [5]:
# unique_names

In [6]:
# df = pd.read_csv('RSI_CCI_MACD.csv')

In [7]:
# df.tail()

In [8]:
def indicator_analysis(stock, df, Start_idx, rows, indicator, threshold, percent_profit, percent_SL):
    new_df = df[df[indicator] >= threshold]
    count = len(new_df)
    indices = new_df.index.tolist()  # Getting each index term

    print(f'For {stock} number of times the {indicator} value is greater than or equal to {threshold}: {count} \n\n')
    # print(df)
    
    for i in range(len(indices)):
      idx = indices[i]   # Got the index value
      date = df.loc[idx, 'TIMESTAMP'] # Got the date for that particular index
      buy_price = df.loc[idx, 'CLOSE'] # Got buy price for that particular index
      target_price = buy_price + (percent_profit/100) * buy_price # Calculated target price for the buy price
      stop_loss_price = buy_price - (percent_SL/100) * buy_price # Calculated stelp loss price for the buy price
      sell_idx = new_df.index.searchsorted(idx+1) # ??
      # print(idx, sell_idx)

      count_tp_breach = 0 # Count number of times tp is breached
      count_sl_breach = 0 # Count number of times sl is breached
      first_time_tp_breached = False # Flag variable to see when tp is first breached
      first_time_sl_breached = False # Flag variable to see when sl is first breached
      actual_sell_price = None # Stores actual sell price
      stock_price_indicator_less_threshold = None # Stores stock value when CCI drops below threshold
      profit_or_loss = None # Stores profit or loss value


      #if i != len(indices) - 1: # If it is not the last time when CCI > threshold -- What if it is the last?
      for j in range(i, len(indices)):  # had i+1
        tp_idx = indices[j]
        sl_idx = indices[j]
        if df.loc[tp_idx, "CLOSE"] >= target_price: # If the value goes above target price
          count_tp_breach += 1 # Count of the number of times the tp is breached
          if first_time_tp_breached == False and first_time_sl_breached == False: # If this is the first time
#                                                                                         # we are seeing such an occurance
            actual_sell_price = df.loc[tp_idx, "CLOSE"] # We sell at this price - at a profit
            first_time_tp_breached = True # Initialize flag variable to True

        elif df.loc[sl_idx, "CLOSE"] <= stop_loss_price: # If the value goes below the stop loss price
          count_sl_breach += 1 # Count of the number of times the sl is breached
          if first_time_tp_breached == False and first_time_sl_breached == False:# If this is the first time
#                                                                                        # we are seeing such an occurance
            
            actual_sell_price = df.loc[sl_idx, "CLOSE"] # We sell that this price - at a loss
            first_time_sl_breached = True # Initialize flag variable to True

      CCI_max = df.loc[idx, "CLOSE"] # Initialize CCI_max with the current
      if idx < len(df)-1:
        for k in range(idx+1, len(df)): # Starting from idx and going through all the remaining indexes of df
          if df.loc[k, indicator] < threshold and not np.isnan(df.loc[k, indicator]):
            stock_price_indicator_less_threshold = df.loc[k, "CLOSE"]
            CCI_Reversal = df.loc[k, indicator]  # What is that??
            break
          elif not np.isnan(df.loc[k, indicator]):
            CCI_max = max(CCI_max, df.loc[k, "CLOSE"])
      else:
        pass

      if actual_sell_price is not None and stock_price_indicator_less_threshold is not None:
        profit_or_loss = stock_price_indicator_less_threshold - actual_sell_price
      else:
        profit_or_loss = 0

      rows.append([date, stock, buy_price, target_price, stop_loss_price, count_tp_breach, count_sl_breach, stock_price_indicator_less_threshold, CCI_max, profit_or_loss])



# #             print(f'***************FOR INDEX {idx}***************')
# #             print(f'Buy Price: {buy_price:.2f}')
# #             print(f'Target Price: {target_price:.2f}')
# #             print(f'Stop Loss Price: {stop_loss_price:.2f}')
# #             print(f'Number of times stock price went above Target Price: {count_tp_breach}')
# #             print(f'Number of times stock price went below Stop Loss Price: {count_sl_breach}')
# #             print(f'Stock Price value when {indicator} became less than {threshold}: {stock_price_indicator_less_threshold}')
# #             print(f'Profit (in positive) or loss (in negative) had we not traded at target price: {profit_or_loss:.2f}\n\n')
# #             print("\n")
            
#             # Start a loop after the index i to see where the CC34 goes below the threshold value
#             # Capture the stock price when CCI34 goes down threshold (j)
#             # Capture the stock price when CCI34 is the max -- it might the value at index i
#         CCI_max = df.loc[Start_idx+i, "CLOSE"]
#         for p in range(i+1, len(df)):
#           if df.loc[Start_idx+p, indicator] < threshold and not np.isnan(df.loc[Start_idx+p, indicator]):
#             CCI_Reversal = df.loc[Start_idx+p, indicator]
#             break
#           elif not np.isnan(df.loc[Start_idx+p, indicator]):
#             CCI_max = max(CCI_max, df.loc[Start_idx+p, "CLOSE"])

#         rows.append([date, stock, buy_price, target_price, stop_loss_price, count_tp_breach, count_sl_breach, stock_price_indicator_less_threshold, CCI_max, profit_or_loss])


In [9]:
count = 0

folder_path = '/content/drive/MyDrive/557 Project - Group 44/CCIDataForAllSymbols/'
save_file_name = folder_path + "Tp_Sp_Profit_or_Loss_CCI.csv"

headers = ['TIMESTAMP', 'Symbol', 'Buy Price', 'Target Price', 'Stop Loss Price', 'Times stock price went above Target Price', 'Times stock price went below Stop Loss Price', 'Stock Price value less than threshold', 'CCI_Max', 'Probable Profit/Loss']
with open(save_file_name, mode='w', newline='') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(headers)
  
  rows = []
  for stock in unique_names:
    df = df_all[df_all["SYMBOL"] == stock]
    # print("COUNT=", len(df) ,count)
    indicator_analysis(stock, df, count, rows, indicator="CCI", threshold=100, percent_profit=20, percent_SL=10)
    count = count + len(df)
    # break

  # print(len(rows))  
  for row in rows:
      writer.writerow(row)

For ZOMATO number of times the CCI value is greater than or equal to 100: 21 


For WIPRO number of times the CCI value is greater than or equal to 100: 273 


For MCDOWELL-N number of times the CCI value is greater than or equal to 100: 273 


For ULTRACEMCO number of times the CCI value is greater than or equal to 100: 230 


For TORNTPHARM number of times the CCI value is greater than or equal to 100: 253 


For TITAN number of times the CCI value is greater than or equal to 100: 304 


For TECHM number of times the CCI value is greater than or equal to 100: 270 


For TATASTEEL number of times the CCI value is greater than or equal to 100: 179 


For TATAPOWER number of times the CCI value is greater than or equal to 100: 219 


For TATAMOTORS number of times the CCI value is greater than or equal to 100: 231 


For TATACONSUM number of times the CCI value is greater than or equal to 100: 128 


For TCS number of times the CCI value is greater than or equal to 100: 292 


For SUNPH

In [10]:
# count = 0

# folder_path = '/content/drive/MyDrive/557 Project - Group 44/RSIDataForAllSymbols/'
# save_file_name = folder_path + "Tp_Sp_Profit_or_Loss_CCI.csv"

# headers = ['TIMESTAMP', 'Symbol', 'Buy Price', 'Target Price', 'Stop Loss Price', 'Times stock price went above Target Price', 'Times stock price went below Stop Loss Price', 'Stock Price value less than threshold', 'RSI_Max', 'Probable Profit/Loss']
# with open(save_file_name, mode='w', newline='') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(headers)
  
#   rows = []
#   for stock in unique_names:
#     df = df_all[df_all["SYMBOL"] == stock]
#     # print("COUNT=", len(df) ,count)
#     indicator_analysis(stock, df, count, rows, indicator="RSI", threshold=70, percent_profit=20, percent_SL=10)
#     count = count + len(df)
#     # break

#   # print(len(rows))  
#   for row in rows:
#       writer.writerow(row)